In [22]:
#VERSAO ITERAVEL DO PROJETO

# Bibliotecas para análise de dados

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.neighbors import DistanceMetric
from itertools import chain, combinations
import random

In [23]:
# Importando arquivo de treino e dividindo os objetos em duas views: mShapeView e mRGBView

train = pd.read_csv('segmentation.test.txt', delimiter=',')
columns = train.columns.tolist()
mShapeView = train.reindex(columns=columns[1:10])
mRGBView = train.reindex(columns=columns[10:20])

In [24]:
#normalizar e calcular as matrizes de dissimilaridade

mShapeViewNorma = normalize(mShapeView, axis=1, norm='l1')
mRGBViewNorma = normalize(mRGBView, axis = 1, norm = 'l1')
dist = DistanceMetric.get_metric('euclidean')

diss_shapeView = dist.pairwise(mShapeViewNorma)
diss_RGBView = dist.pairwise(mRGBViewNorma)
#type(diss_shapeView)
#np.shape(diss_shapeView)

In [25]:
matrizes = [diss_shapeView, diss_RGBView]
np.shape(matrizes[1])

(2100, 2100)

In [26]:
#logica do algoritmo
#variaveis globais
NUM_CLUST = 7
NUM_OBJ = 2100
NUM_PESOS = 2
NUM_REP = 3
VAL_MAX = 10000000
pesos = np.ones(2)
matrizes = [diss_shapeView, diss_RGBView]


random.seed(100)


E = np.arange(0,2100).tolist() #representa o indice de cada objeto, se acessado pela matriz de dissimilaridade
#nesse momento serve para calclar os subconjuntos de card 'q' usados na logica dos representantes

#OBS
#para a escolha dos prototipos a ideia eh testar o matching global dos elementos de um cado cluster Ck
#com um canditado e_h a representante, onde h varia de 0 a n, n sendo o numero de elementos do conjunto E
#depois ordena a lista que contem o valor dos matchings e pega os 3 melhores (melhor signifcand)
#A distancia para os 3 elementos dos representantes eh utilizada apenas na etape de definir a nova particao

#definir os prototipos; inicializar de forma aletoria
prototipos = []
#definir a particao
particao = [] #particao eh composta por uma lista de clusters, cada um contendo os indices dos elementos
#passo 1 - inicializacao
prototipos_novos = [] #prototipos G_k_t do artigo



#------------------funcoes auxiliares do estagio 2-------------------------------------
#o termo referente ao match global dos elementos dos clusters com os prototipos,
#ponderado pela matriz acessada pelo indice. Esse termo eh util na computacao
#do melhor peso e eh retornado pela funcao abaixo
def match_global(indice):     #(prototipos, particao, indice):
    resultado = np.zeros(1) #tem que ver se esse resultado consegue fazera divisao corretamente
    
    
    
    for k in range(0, NUM_CLUST):

        for e_i in particao[k]:

            for e in prototipos_novos[k]: #aqui depende de prototipos novos como variavel global
                resultado[0] += matrizes[indice][e_i][e] #precisa de indice para as matrizes aqui
            #nesse momento match_prot contem o match global de e_i com o prototipo, ponderado pelas duas
            #matrizes de dissimilaridade    
    
    
    
    return resultado


#retorna o lambda referente ao indice passado, representado como lambda j no artigo
def melhor_peso(indice):
    resultado = np.ones(2) #um para o numerador e outro para o denominador
    resultado_aux = 0.0
    
    for i in range(0, NUM_PESOS):
        resultado_aux = match_global(i) #aqui considera todas as matrizes em questao
        resultado[0] *= resultado_aux;
    
    resultado[1] = match_global(indice) #denominador
    
    resultado[0] = resultado[0]**(1/NUM_PESOS) #numerador
    
    
    
    return (resultado[0] / resultado[1])


#---------------------------------------------------------------------------------------------



#inicializacao dos representantes
for i in range(0,NUM_CLUST):
    prototipos.append([random.randint(i, 10 + i + i*150),random.randint(i, 50  + i + i*150),random.randint(i,150 + i +  i*150)])
    particao.append([])

    

#Nesse momento o vetor E passa a armazenar o indice do cluster k ao qual o elemento indexado pertence
for i in range(0, NUM_OBJ):
    indice_atual = 0 #indice do cluster que minimiza o criterio J para o elemento e_i
    criterio_atual = VAL_MAX #criterio atual para elemento e_im; quer minimizar o criterio
    
    for index_prot, prot in enumerate(prototipos, start = 0):
        #indice_prop = 0
        match_prot = 0 #calcula a dissimilaridade de e_i com cada elemento do prototipo
        for j, val_peso in enumerate(pesos, start = 0): #num de pesos eh igual do de matrizes
            
            for e in prot:
                match_prot += val_peso*(matrizes[j][e][i]) #precisa de indice para as matrizes aqui
        #nesse momento match_prot contem o match global de e_i com o prototipo, ponderado pelas duas
        #matrizes de dissimilaridade
        if(match_prot < criterio_atual):
            criterio_atual = match_prot
            indice_atual = index_prot
    #espera chegar aqui com o index do melhor prototipo para e_i. Logo, pode adiciona-lo ao cluster
    particao[indice_atual].append(i) #i representa o index do elemento
    E[i] = indice_atual
    



In [31]:
#-------------------------------------parte iteravel do algoritmo------------------------------
for cont_iteracao in range(0, 15): #usar uma constante aqui depois
    #ESTAGIO 1
    #computar os melhores prototipos; 
    #Aqui usa a nova lista de prototipos
    
    prototipos_novos = []
    
    for i in range(0, NUM_CLUST):
        prototipos_novos.append([])


    for k in range(0,NUM_CLUST):
        Rep = []

        for h in range(0,NUM_OBJ):
            match_candidato = 0 #usa para armazenar o match dos elementos do cluster k
            #com o candidato e_h
            for j, val_peso in enumerate(pesos, start = 0): #iterador para pesos e matrizes

                for obj in particao[k]: #para cada elemento do cluster k
                    match_candidato += val_peso*(matrizes[j][obj][h]) #match de um obj do cluster k
                    #com o candidato a representante h, ponderado pelo peso j na matriz j
            #computou o match do canditato com todos os elementos do cluster
            Rep.append([match_candidato, h]) #insere match na primeira posicao pois ordena por ele

        #canditados h com matchs calculados, ordena pelo match e pega os 3 melhores 
        Rep.sort()

        for ind in range(0,3): #usar NUM_REP aqui
            prototipos_novos[k].append(Rep[ind][1]) #tem que pegar a posicao 1 do Rep que contem o indice
    
    #ESTAGIO 2
    #considera os novos prototipos calculados anteriormente e a particao anterior ou incial
    #assume globais todas as variaveis nao declaradas na funcao ou como parametros
    #usa as funcoes auxiliares para organizar melhor a logica
    #essa eh a parte do codigo REALMENTE responsavel por calcular os pesos das matrizes
    for indiz in range(0, NUM_PESOS):
        pesos[indiz] = melhor_peso(indiz)

    #TERCEIRA PARTE
    #alterar o cluster a que pertence um objeto caso se adeque melhor a nova configuracao de 
    #pesos e prototipos calculados

    #ver necessidade de criar um clone da particao ou se simplesmente vai excluindo e adicionando
    #elementos nos clusters da particao atual
    #o que importa eh achar o melhor match com relacao aos novos prototipos  e pesos
    #calculados no passo 1

    for i in range(0, NUM_OBJ): #para cada elemento de E
        indice_novo = E[i] #indice do cluster atual do elemento i eh E[i], mas inicia o novo igual
        criterio_atual = VAL_MAX #criterio atual para elemento e_im; cuidado, quer minimizar o criterio
        for index_prot, prot in enumerate(prototipos_novos, start = 0):
            #indice_prop = 0
            match_prot = 0 #calcula a dissimilaridade de e_i com cada elemento do prototipo
            for j, val_peso in enumerate(pesos, start = 0): #num de pesos eh igual do de matrizes

                for e in prot:
                    match_prot += val_peso*(matrizes[j][e][i]) #precisa de indice para as matrizes aqui
            #nesse momento match_prot contem o match global de e_i com o prototipo, ponderado pelas duas
            #matrizes de dissimilaridade
            if(match_prot < criterio_atual):
                criterio_atual = match_prot
                indice_novo = index_prot
        #espera chegar aqui com o index do melhor prototipo para e_i.
        #Agora realiza alteracoes na particao
        if(indice_novo != E[i]):
            particao[E[i]].remove(i) #remove da particao antiga
            particao[indice_novo].append(i) #add no novo cluster
            E[i] = indice_novo #atualiza indice do elemento e_i

        #particao[indice_atual].append(i) #i representa o index do elemento
        
    print(cont_iteracao)       


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [32]:
for i in particao:
    print(len(i))

129
306
415
168
645
389
48


In [17]:
for i in particao:
    print(len(i))

123
225
495
179
671
356
51


In [33]:
#teste do assignment inicial dos objetos ao prototipo que otimiza melhor o criterio
total = 0
for i in range(0, NUM_CLUST):
    total += len(particao[i])
total

2100

In [35]:
E[2070:2090]

[5, 4, 1, 1, 1, 1, 1, 1, 4, 5, 4, 4, 4, 5, 5, 5, 1, 4, 1, 5]